# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [2]:
!pip install -Uqqq openai tiktoken wandb tenacity

In [105]:
import os
import random
from pathlib import Path
import openai
import tiktoken
from pprint import pprint
from IPython.display import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

In [6]:
# We will add a retry behavior in case we hit the API rate limit

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [91]:
# MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4"

In [117]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL,
        messages=messages,
        n = 5,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

"Hello, as a W&B user, I am having trouble understanding how to properly use the W&B Dashboard for visualizing my experimentation logs. Can you provide some guidance and best practices?"

"Hello, as a W&B user, I am having difficulty figuring out how to visualize my model's training progress using the dashboard. Can you please guide me through the process or provide some resources to assist me in setting up this feature? Thank you!"

"Hello, I'm a W&B user and I've recently encountered an issue while trying to log my training data. Can you please guide me through the steps to ensure I'm doing it correctly? Thanks!"

"Hello, I am a W&B user, and I am having difficulty in setting up my project. Could you guide me on how to correctly configure the W&B integration with my machine learning code to track experiments and visualize results? Thanks!"

Hi, I'm a user of W&B and I have a question about integrating it with my PyTorch project. Can you please guide me on how to implement W&B logging in my training and validation processes? Thank you!

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [114]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: 
"How do I perform large rearrangements in web ui table efficiently? Dragging columns to the wanted positions one-by-one is very slow."

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [115]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user
Below you will find a few examples of real user queries:
Is there any way to name different experiments within the same run? W&B process takes some time to init and to finish, so I am losing efficiency in the experiment tracking
What are the risk of supplying proprietary data into a third party cloud provider such as weights and biases?
how can i do .tight_layout() on my seaborn heatmap
Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [97]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

("Hello, I'm using W&B for my experiments, and I encountered an issue "
 'regarding authentication on a cloud service. Our team has a team account, '
 'and we use a client application to initiate the training on the cloud '
 "server. However, the cloud server requires authentication. I'm wondering if "
 'each user has to authenticate personally, or is there a way to use our team '
 'account to enable authentication for all users, even if they are not logged '
 'in to the server machines? Thanks for your help!')
("Hello! I've been using W&B to track my experiments, and I'm running multiple "
 "training jobs simultaneously. However, I'm not sure how to properly organize "
 'the runs and visualize their results in the dashboard. Is there an efficient '
 'way to group related runs and filter them easily on the W&B interface?')
('Hey there, I recently started using W&B for my machine learning experiments, '
 'and I have a question regarding runs and authentication. When I am training '
 'my

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [98]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [99]:
tokenizer = tiktoken.encoding_for_model(MODEL)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[4179, 365, 1206, 2596, 2940, 537, 956, 803, 1644, 2529, 2093]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [100]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [101]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [102]:
pprint(generation_prompt)

('Generate a support question from a W&B user\n'
 'The question should be answerable by provided fragment of W&B '
 'documentation.\n'
 'Below you will find a fragment of W&B documentation:\n'
 ' import WandbLogger\n'
 'from pytorch_lightning import Trainer\n'
 '\n'
 'wandb_logger = WandbLogger()\n'
 'trainer = Trainer(logger=wandb_logger)\n'
 '```\n'
 '\n'
 '![Interactive dashboards accessible anywhere, and '
 'more!](@site/static/images/integrations/n6P7K4M.gif)\n'
 '\n'
 '## Sign up and Log in to wandb\n'
 '\n'
 'a) [**Sign up**](https://wandb.ai/site) for a free account\n'
 '\n'
 'b) Pip install the `wandb` library\n'
 '\n'
 "c) To login in your training script, you'll need to be signed in to you "
 'account at www.wandb.ai, then **you will find your API key on the** '
 '[**Authorize page**](https://wandb.ai/authorize)**.**\n'
 '\n'
 'If you are using Weights and Biases for the first time you might want to '
 'check out our [**quickstart**](../../quickstart.md)****\n'
 '\n'
 '<Tabs

In [103]:
generate_and_print(system_prompt, generation_prompt)

('Question: How do I use the WandbLogger in PyTorch Lightning to log metrics, '
 'model weights, and media?\n'
 '\n'
 'Fragment from W&B Documentation:\n'
 '\n'
 '```python\n'
 'import WandbLogger\n'
 'from pytorch_lightning import Trainer\n'
 '\n'
 'wandb_logger = WandbLogger()\n'
 'trainer = Trainer(logger=wandb_logger)\n'
 '```')
("Question: How can I use the WandbLogger with PyTorch Lightning's Trainer to "
 'log my training metrics and model weights with Weights & Biases?')
("Question: How can I use the WandbLogger with PyTorch Lightning's Trainer to "
 'log metrics and model weights?\n'
 '\n'
 'Documentation Fragment:\n'
 '\n'
 '```\n'
 'import WandbLogger\n'
 'from pytorch_lightning import Trainer\n'
 '\n'
 'wandb_logger = WandbLogger()\n'
 'trainer = Trainer(logger=wandb_logger)\n'
 '```')
('Question: How do I use W&B with PyTorch Lightning, and where can I find my '
 'API key?')
('Question: How can I use the WandbLogger class from PyTorch Lightning to log '
 'metrics, model we

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [83]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [85]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return prompt

generation_prompt = generate_context_prompt(chunk)

In [86]:
pprint(generation_prompt)

('You are a creative assistant with the goal to generate a synthetic dataset '
 'of Weights & Biases (W&B) user questions.\n'
 "W&B users are asking these questions to a bot, so they don't know the answer "
 "and their questions are grounded in what they're trying to achieve. \n"
 'We are interested in questions that can be answered by W&B documentation. \n'
 "But the users don't have access to this documentation, so you need to "
 "imagine what they're trying to do and use according language. \n"
 'Here are some examples of real user questions, you will be judged by how '
 'well you match this distribution.\n'
 '***\n'
 'when I try ```api = wandb.Api()\n'
 'project = api.project("<entity>/<project_name>")\n'
 "sweeps = project.sweeps()) ``` it gives me ``` 'NoneType' object is not "
 'subscriptable.``` Do you know why?\n'
 'how do you log a dataset?\n'
 'how do i prevent my wandb.Image from cutting off the ticklabels?\n'
 '***\n'
 'In the next step, you will read a fragment of W&B doc

In [87]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
        response = completion_with_backoff(
            model=model,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [88]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [89]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

("A user is trying to log their model metrics and hyperparameters using W&B. They have read the documentation and know that they can log their metrics when using `WandbLogger` by calling `self.log('my_metric_name', metric_vale)`. However, they are unsure about how to use the `self.log()` method within their code to log their metrics and hyperparameters.\n",
 "Hi there! I'm new to W&B and PyTorch Lightning, and I'm trying to use the `WandbLogger` to log my model metrics and hyperparameters during training. I read the documentation on how to log metrics, but I'm still a bit confused on how to use the `self.log()` method in my code. Can you help me out?\n",
 'Absolutely! In order to use the `self.log()` method to log your model metrics and hyperparameters during training with `WandbLogger`, first you need to define your `LightningModule` to log your metrics and hyperparameters as shown in the documentation. Then, you can call `self.log(\'my_metric_name\', metric_value)` within your `Light

In [92]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# log df as a table to W&B
wandb.log({"generated_examples": wandb.Table(dataframe=pd.DataFrame(parsed_generations))})

In [93]:
wandb.finish()

In [101]:
parsed_generations[8]['question']

'\nHow can I share my report with my team members in a public project?\n'

In [103]:
pprint(parsed_generations[8]['answer'])

('\n'
 'To share a report with your team members in a public project, select the '
 'Share button on the upper right hand corner of the report. You can either '
 'provide an email account or copy the magic link. Users invited by email will '
 'need to log into Weights & Biases to view the report. Users who are given a '
 'magic link do not need to log in to Weights & Biases to view the report. '
 "It's important to note that shared reports are view-only, and only the "
 'administrator or the member who created the report can toggle permissions '
 'between edit or view access for other team members.')
